In [ ]:
# Program Sistem Rekomendasi Laptop menggunakan Backward Chaining dan Certainty Factor

# =================================================================
# 1. BASIS PENGETAHUAN (KNOWLEDGE BASE)
# =================================================================

# Definisi aturan dengan kondisi, goal, dan CF rule
RULES = {
    'R1': {'conditions': ['gaming', 'budget_tinggi', 'portabilitas_rendah'],
           'goal': 'laptop_gaming',
           'cf_rule': 0.9},
    'R2': {'conditions': ['desain_grafis', 'layar_bagus', 'ram_besar'],
           'goal': 'laptop_creator',
           'cf_rule': 0.85},
    'R3': {'conditions': ['office', 'battery_awet', 'harga_terjangkau'],
           'goal': 'laptop_bisnis',
           'cf_rule': 0.8},
    'R4': {'conditions': ['ringan', 'battery_awet', 'harga_terjangkau'],
           'goal': 'laptop_student',
           'cf_rule': 0.75},
    'R5': {'conditions': ['programming', 'cpu_kuat', 'ram_besar'],
           'goal': 'laptop_developer',
           'cf_rule': 0.88}
}

# Menyimpan fakta dan nilai CF dari user
FACTS_CF = {}

# Membuat mapping antara goal dan rule terkait
GOAL_TO_RULES = {}
for rule_name, rule_data in RULES.items():
    goal = rule_data['goal']
    if goal not in GOAL_TO_RULES:
        GOAL_TO_RULES[goal] = []
    GOAL_TO_RULES[goal].append(rule_name)

# =================================================================
# 2. FUNGSI PERHITUNGAN CERTAINTY FACTOR
# =================================================================

# Menggabungkan dua nilai CF berdasarkan formula kombinasi evidence
def gabung_cf(cf_old, cf_new):
    """Menggabungkan dua CF: CF_gabungan = CF_old + CF_new * (1 - CF_old)"""
    return cf_old + cf_new * (1 - cf_old)

# =================================================================
# 3. FUNGSI BACKWARD CHAINING
# =================================================================

# Mengambil input keyakinan user untuk suatu fakta
def get_cf_user(fact):
    """
    Mengambil CF user untuk suatu fakta.
    Jika fakta belum diketahui, akan ditanyakan ke user.
    """
    if fact in FACTS_CF:
        return FACTS_CF[fact]

    label = fact.replace('_', ' ').title()

    while True:
        try:
            cf = input(f"CF keyakinan Anda pada '{label}' (0.0 - 1.0): ")
            if not cf:
                cf_value = 0.0
            else:
                cf_value = float(cf)

            if 0.0 <= cf_value <= 1.0:
                FACTS_CF[fact] = cf_value
                return cf_value
            else:
                print("CF harus antara 0.0 dan 1.0. Coba lagi.")
        except ValueError:
            print("Input tidak valid. Harap masukkan angka.")

# Menghitung CF hipotesis berdasarkan kondisi dan rule tertentu
def apply_rule(rule_name, rule_data):
    """
    Menerapkan satu rule, menghitung CF kondisi gabungan dan CF hipotesis.
    """
    conditions = rule_data['conditions']
    cf_rule = rule_data['cf_rule']

    # Mengumpulkan CF dari setiap kondisi
    cf_conditions = []
    for condition in conditions:
        cf_conditions.append(get_cf_user(condition))

    # Mengambil nilai minimum (AND) dari semua kondisi
    cf_kondisi_gabungan = min(cf_conditions)

    # Menghitung CF hipotesis
    cf_hipotesis = cf_kondisi_gabungan * cf_rule

    return cf_hipotesis

# Melakukan backward chaining untuk menemukan CF dari goal tertentu
def backward_chaining(goal):
    """
    Melakukan penelusuran backward chaining berdasarkan goal.
    """
    if goal not in GOAL_TO_RULES:
        return 0.0

    final_cf = 0.0

    for rule_name in GOAL_TO_RULES[goal]:
        rule_data = RULES[rule_name]
        cf_new = apply_rule(rule_name, rule_data)

        if final_cf == 0.0:
            final_cf = cf_new
        else:
            final_cf = gabung_cf(final_cf, cf_new)

    return round(final_cf, 4)

# =================================================================
# 4. PROGRAM UTAMA
# =================================================================

# Menjalankan keseluruhan proses backward chaining dan menampilkan hasil
def main():
    """Menjalankan proses rekomendasi laptop berdasarkan input user."""
    print("===== SISTEM REKOMENDASI LAPTOP (Backward Chaining + CF) =====")
    print("Silakan tentukan tingkat keyakinan (0.0 - 1.0) Anda pada setiap kriteria.")

    all_goals = list(GOAL_TO_RULES.keys())
    hasil_rekomendasi = {}

    # Proses evaluasi setiap jenis laptop
    print("\n--- Proses Evaluasi Kriteria ---")
    for goal in all_goals:
        print(f"\n[EVALUASI UNTUK {goal.upper().replace('_', ' ')}]")
        cf_result = backward_chaining(goal)
        hasil_rekomendasi[goal] = cf_result

    # Menampilkan hasil akhir dalam bentuk tabel
    print("\n" + "="*50)
    print("            HASIL AKHIR REKOMENDASI")
    print("="*50)

    sorted_results = sorted(hasil_rekomendasi.items(), key=lambda item: item[1], reverse=True)

    if not sorted_results or sorted_results[0][1] == 0.0:
        print("Tidak ada rekomendasi yang kuat (semua CF = 0.0).")
        return

    print(f"{'Rekomendasi':<20} {'CF Final':<10} {'Keyakinan'}")
    print("-" * 50)
    for goal, cf in sorted_results:
        percentage = f"{(cf * 100):.2f} %"
        print(f"{goal.replace('_', ' ').title():<20} {cf:<10.4f} {percentage}")

    best_goal = sorted_results[0][0]
    best_cf = sorted_results[0][1]

    print("-" * 50)
    print(f"REKOMENDASI TERKUAT: {best_goal.replace('_', ' ').upper()}")
    print(f"Certainty Factor (CF): {best_cf} ({(best_cf * 100):.2f} %)")
    print("="*50)


if __name__ == "__main__":
    main()
